## SQL Query Hands-on

**Domain Retail sales.**

- Create two tables: `sales` and `products`
- Populate them with sample data
- Run analytical queries with JOIN, GROUP BY, CTE, window functions, etc.

In [0]:
-- Sales Table
DROP TABLE IF EXISTS sales;
CREATE TABLE sales (
  sale_id INT,
  product_id INT,
  quantity INT,
  unit_price DOUBLE,
  sale_date DATE
);

-- Products Table
DROP TABLE IF EXISTS products;
CREATE TABLE products (
  product_id INT,
  product_name STRING,
  category STRING
);


**Insert Sample Data**

In [0]:
INSERT INTO products VALUES
(1, 'Laptop Pro', 'Electronics'),
(2, 'Smartphone X', 'Electronics'),
(3, 'Desk Chair', 'Furniture'),
(4, 'Office Desk', 'Furniture'),
(5, 'Wireless Mouse', 'Accessories');

INSERT INTO sales VALUES
(101, 1, 3, 1200.00, '2023-07-01'),
(102, 2, 5, 800.00, '2023-07-02'),
(103, 1, 2, 1150.00, '2023-07-03'),
(104, 3, 7, 150.00, '2023-07-04'),
(105, 4, 1, 300.00, '2023-07-05'),
(106, 2, 3, 790.00, '2023-07-06'),
(107, 5, 10, 50.00, '2023-07-07'),
(108, 3, 4, 155.00, '2023-07-08'),
(109, 4, 2, 310.00, '2023-07-09'),
(110, 5, 6, 55.00, '2023-07-10');

### Query 1 – Total Sales Amount Per Category

In [0]:
SELECT
  p.category,
  SUM(s.quantity * s.unit_price) AS total_sales
FROM sales s
JOIN products p
  ON s.product_id = p.product_id
GROUP BY p.category;

### Query 2 – Top 3 Sales by Quantity

In [0]:
SELECT *
FROM sales
ORDER BY quantity DESC
LIMIT 3;

###  Query 3 – Average Sale Price per Product

In [0]:
SELECT
  p.product_name,
  ROUND(AVG(s.unit_price), 2) AS avg_price
FROM sales s
JOIN products p
  ON s.product_id = p.product_id
GROUP BY p.product_name;


### Query 4 – CTE for Sales in July 2023

In [0]:
WITH july_sales AS (
  SELECT
    s.*,
    p.product_name
  FROM sales s
  JOIN products p
    ON s.product_id = p.product_id
  WHERE s.sale_date BETWEEN '2023-07-01' AND '2023-07-31'
)
SELECT
  product_id,
  product_name,
  SUM(quantity) AS total_units
FROM july_sales
GROUP BY product_id, product_name
ORDER BY product_id;


### Query 5 – Running Total of Sales Amount by Date

In [0]:
SELECT
  sale_date,
  SUM(quantity * unit_price) OVER (
    ORDER BY sale_date
    ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
  ) AS running_total
FROM sales
ORDER BY sale_date;

### Query 6 – Dense Rank of Products by Quantity Sold

In [0]:
SELECT
  p.product_name,
  SUM(s.quantity) AS total_quantity,
  DENSE_RANK() OVER (
    ORDER BY SUM(s.quantity) DESC
  ) AS quantity_rank
FROM sales s
JOIN products p
  ON s.product_id = p.product_id
GROUP BY p.product_name;

### Query 7 – Join and Filter Electronics Sales Only

In [0]:
SELECT
  s.sale_id,
  p.product_name,
  s.quantity,
  s.unit_price
FROM sales s
JOIN products p
  ON s.product_id = p.product_id
WHERE p.category = 'Electronics';

### Query 8 – Find Max and Min Sale Date per Product

In [0]:
SELECT
  p.product_name,
  MAX(s.sale_date) AS last_sale,
  MIN(s.sale_date) AS first_sale
FROM sales s
JOIN products p
  ON s.product_id = p.product_id
GROUP BY p.product_name;

### Query 9 – Average Quantity Sold per Category

In [0]:
SELECT
  p.category,
  ROUND(AVG(s.quantity),2) AS avg_quantity
FROM sales s
JOIN products p
  ON s.product_id = p.product_id
GROUP BY p.category;

### Query 10 – Sale Amount with Row Number per Category

In [0]:
SELECT
  p.category,
  s.sale_id,
  s.quantity * s.unit_price AS sale_amount,
  ROW_NUMBER() OVER (
    PARTITION BY p.category
    ORDER BY s.sale_date
  ) AS row_num
FROM sales s
JOIN products p
  ON s.product_id = p.product_id;

## Add a Customers table

In [0]:
DROP TABLE IF EXISTS customers;
CREATE TABLE customers (
  customer_id INT,
  customer_name STRING,
  region STRING
);

**Insert Sample Customer Data**

In [0]:
INSERT INTO customers VALUES
(1, 'Alice Johnson', 'North'),
(2, 'Bob Smith', 'South'),
(3, 'Charlie Lee', 'East'),
(4, 'Diana Prince', 'West'),
(5, 'Ethan Brown', 'North');

**Create new sales table to include this column customer_id**

In [0]:
-- Recreate for clarity
DROP TABLE IF EXISTS sales;
CREATE TABLE sales (
  sale_id INT,
  product_id INT,
  customer_id INT,
  quantity INT,
  unit_price DOUBLE,
  sale_date DATE
);

INSERT INTO sales VALUES
(101, 1, 1, 3, 1200.00, '2023-07-01'),
(102, 2, 2, 5, 800.00, '2023-07-02'),
(103, 1, 3, 2, 1150.00, '2023-07-03'),
(104, 3, 4, 7, 150.00, '2023-07-04'),
(105, 4, 5, 1, 300.00, '2023-07-05'),
(106, 2, 1, 3, 790.00, '2023-07-06'),
(107, 5, 2, 10, 50.00, '2023-07-07'),
(108, 3, 3, 4, 155.00, '2023-07-08'),
(109, 4, 4, 2, 310.00, '2023-07-09'),
(110, 5, 5, 6, 55.00, '2023-07-10');

### Total Sales by Customer and Region

In [0]:
SELECT
  c.customer_name,
  c.region,
  SUM(s.quantity * s.unit_price) AS total_sales
FROM sales s
JOIN customers c
  ON s.customer_id = c.customer_id
GROUP BY c.customer_name, c.region;

### Top 2 Products per Region by Quantity Sold

In [0]:
WITH sales_with_details AS (
  SELECT
    s.sale_id,
    s.product_id,
    s.customer_id,
    s.quantity,
    c.region,
    p.product_name
  FROM sales s
  JOIN customers c
    ON s.customer_id = c.customer_id
  JOIN products p
    ON s.product_id = p.product_id
),
aggregated_sales AS (
  SELECT
    region,
    product_name,
    SUM(quantity) AS total_quantity
  FROM sales_with_details
  GROUP BY region, product_name
),
ranked_products AS (
  SELECT
    region,
    product_name,
    total_quantity,
    RANK() OVER (
      PARTITION BY region
      ORDER BY total_quantity DESC
    ) AS rank_in_region
  FROM aggregated_sales
)
SELECT
  region,
  product_name,
  total_quantity,
  rank_in_region
FROM ranked_products
WHERE total_quantity > 0;


### Monthly Sales Summary

In [0]:
WITH monthly_sales AS (
  SELECT
    DATE_FORMAT(sale_date, 'yyyy-MM') AS month,
    SUM(quantity * unit_price) AS total_sales
  FROM sales
  GROUP BY DATE_FORMAT(sale_date, 'yyyy-MM')
)
SELECT *
FROM monthly_sales
ORDER BY month;


### Customer Purchase Frequency

In [0]:
WITH customer_purchase_counts AS (
  SELECT
    s.customer_id,
    c.customer_name,
    COUNT(*) AS total_purchases
  FROM sales s
  JOIN customers c
    ON s.customer_id = c.customer_id
  GROUP BY s.customer_id, c.customer_name
)
SELECT
  customer_id,
  customer_name,
  total_purchases
FROM customer_purchase_counts
ORDER BY total_purchases DESC;


### Average Sale Value per Region

In [0]:
SELECT
  c.region,
  ROUND(AVG(s.quantity * s.unit_price),2) AS avg_sale_value
FROM sales s
JOIN customers c
  ON s.customer_id = c.customer_id
GROUP BY c.region;

### Last Purchase Date per Customer

In [0]:
SELECT
  customer_id,
  MAX(sale_date) AS last_purchase_date
FROM sales
GROUP BY customer_id;

### Products Never Sold

In [0]:
SELECT
  p.product_id,
  p.product_name
FROM products p
LEFT JOIN sales s
  ON p.product_id = s.product_id
WHERE s.product_id IS NULL;

In [0]:
INSERT INTO products
VALUES
(6, 'Ergonomic Keyboard', 'Accessories');

### Highest Single Sale per Customer

In [0]:
SELECT
  customer_id,
  MAX(quantity * unit_price) AS max_sale_amount
FROM sales
GROUP BY customer_id;